## 卷积神经网络（Convolutional Neural Network, CNN）

## 实现一个狗品种识别算法App

---
<a id='step0'></a>
##  导入数据集

### 导入狗数据集
使用 scikit-learn 库中的 `load_files` 函数来获取一些变量：
- `train_files`, `valid_files`, `test_files` - 包含图像的文件路径的numpy数组
- `train_targets`, `valid_targets`, `test_targets` - 包含独热编码分类标签的numpy数组
- `dog_names` - 由字符串构成的与标签相对应的狗的种类

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob


def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets


train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')


dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]


print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


### 导入人脸数据集

导入人脸图像数据集，文件所在路径存储在名为 `human_files` 的 numpy 数组。

In [3]:
import random
random.seed(8675309)

human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)

print('There are %d total human images.' % len(human_files))

There are 13233 total human images.


---

<a id='step3'></a>

## 创建一个CNN来分类狗品种


随机分类将得到一个非常低的结果：不考虑品种略有失衡的影响，随机猜测到正确品种的概率是1/133，相对应的准确率是低于1%的。



### 数据预处理


通过对每张图像的像素值除以255，对图像实现了归一化处理。

In [10]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# Keras中的数据预处理过程
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████| 836/836 [00:56<00:00, 14.67it/s]


### 模型架构


In [13]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()


model.add(Conv2D(filters = 16,kernel_size = 2,padding = 'valid',activation = 'relu',input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(filters = 32,kernel_size = 2,padding = 'valid',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(filters = 64,kernel_size = 2,padding = 'valid',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(GlobalAveragePooling2D(data_format = None))
model.add(Dense(133,activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 54, 54, 64)        8256      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

In [12]:
## 编译模型
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

---

## 训练模型


In [13]:
from keras.callbacks import ModelCheckpoint  


epochs = 5


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/5
6680/6680 [==============================] - 901s - loss: 4.8853 - acc: 0.0079 - val_loss: 4.8713 - val_acc: 0.0108
Epoch 2/5
6680/6680 [==============================] - 957s - loss: 4.8649 - acc: 0.0120 - val_loss: 4.8489 - val_acc: 0.0132
Epoch 3/5
6680/6680 [==============================] - 957s - loss: 4.8240 - acc: 0.0175 - val_loss: 4.8053 - val_acc: 0.0204
Epoch 4/5
6680/6680 [==============================] - 816s - loss: 4.7745 - acc: 0.0196 - val_loss: 4.7824 - val_acc: 0.0168
Epoch 5/5
6680/6680 [==============================] - 847s - loss: 4.7399 - acc: 0.0229 - val_loss: 4.7564 - val_acc: 0.0204


In [14]:
## 加载具有最好验证loss的模型

model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### 测试模型

在狗图像的测试数据集上试用的模型。

In [15]:
# 获取测试数据集中每一个图像所预测的狗品种的index
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# 报告测试准确率
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 3.3493%


---
<a id='step4'></a>
## 使用一个CNN来区分狗的品种


使用 迁移学习（Transfer Learning）的方法，能帮助我们在不损失准确率的情况下大大减少训练时间。

### 得到从图像中提取的特征向量（Bottleneck Features）

In [15]:
bottleneck_features = np.load('bottleneck_features/DogVGG16Data.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']

### 模型架构

该模型使用预训练的 VGG-16 模型作为固定的图像特征提取器，其中 VGG-16 最后一层卷积层的输出被直接输入到我们的模型。只需要添加一个全局平均池化层以及一个全连接层，其中全连接层使用 softmax 激活函数，对每一个狗的种类都包含一个节点。

In [16]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(133, activation='softmax'))

VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               68229     
Total params: 68,229.0
Trainable params: 68,229.0
Non-trainable params: 0.0
_________________________________________________________________


In [17]:
## 编译模型

VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [18]:
## 训练模型

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, 
          validation_data=(valid_VGG16, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)



Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 8s - loss: 12.4018 - acc: 0.1260 - val_loss: 10.8847 - val_acc: 0.2144
Epoch 2/20
6680/6680 [==============================] - 3s - loss: 10.2301 - acc: 0.2762 - val_loss: 10.0997 - val_acc: 0.2874
Epoch 3/20
6680/6680 [==============================] - 3s - loss: 9.7336 - acc: 0.3418 - val_loss: 10.1020 - val_acc: 0.3054
Epoch 4/20
6680/6680 [==============================] - 4s - loss: 9.4873 - acc: 0.3659 - val_loss: 9.7692 - val_acc: 0.3126
Epoch 5/20
6680/6680 [==============================] - 3s - loss: 9.1677 - acc: 0.3940 - val_loss: 9.5061 - val_acc: 0.3425
Epoch 6/20
6680/6680 [==============================] - 3s - loss: 8.9448 - acc: 0.4162

In [19]:
## 加载具有最好验证loss的模型

VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### 测试模型
现在，我们可以测试此CNN在狗图像测试数据集中识别品种的效果如何。我们在下方打印出测试准确率。

In [20]:
# 获取测试数据集中每一个图像所预测的狗品种的index
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# 报告测试准确率
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 40.9091%


### 使用模型预测狗的品种

In [21]:
from extract_bottleneck_features import *

def VGG16_predict_breed(img_path):
    # 提取bottleneck特征
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    # 获取预测向量
    predicted_vector = VGG16_model.predict(bottleneck_feature)
    # 返回此模型预测的狗的品种
    return dog_names[np.argmax(predicted_vector)]

## 使用InceptionV3模型进行迁移学习

In [11]:

import numpy as np
bottleneck_features = np.load('bottleneck_features/DogInceptionV3Data.npz')
train_InceptionV3 = bottleneck_features['train']
valid_InceptionV3 = bottleneck_features['valid']
test_InceptionV3 = bottleneck_features['test']

### 模型架构

建立一个CNN来分类狗品种。


In [14]:

InceptionV3_model = Sequential()
InceptionV3_model.add(GlobalAveragePooling2D(input_shape = train_InceptionV3.shape[1:]))
InceptionV3_model.add(Dense(133,activation = 'softmax'))

InceptionV3_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               272517    
Total params: 272,517.0
Trainable params: 272,517.0
Non-trainable params: 0.0
_________________________________________________________________


In [15]:

InceptionV3_model.compile(loss = 'categorical_crossentropy',optimizer = 'rmsprop',metrics = ['accuracy'])

---

### 训练模型



In [17]:
from keras.callbacks import ModelCheckpoint

In [ ]:

checkpointer = ModelCheckpoint(filepath = 'saved_models/weights.best.InceptionV3.hdf5',verbose = 1,save_best_only = True)

InceptionV3_model.fit(train_InceptionV3,train_targets,
                     validation_data = (valid_InceptionV3,valid_targets),
                     epochs = 20,batch_size = 20,callbacks = [checkpointer],verbose = 1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 18s - loss: 1.1583 - acc: 0.7106 - val_loss: 0.6882 - val_acc: 0.8072
Epoch 2/20
6680/6680 [==============================] - 11s - loss: 0.4771 - acc: 0.8530 - val_loss: 0.6574 - val_acc: 0.8407
Epoch 3/20
6680/6680 [==============================] - 9s - loss: 0.0465 - acc: 0.9853 - val_loss: 0.8646 - val_acc: 0.8635
Epoch 18/20
6680/6680 [==============================] - 12s - loss: 0.0369 - acc: 0.9885 - val_loss: 0.9792 - val_acc: 0.8563
Epoch 19/20
6680/6680 [==============================] - 12s - loss: 0.0342 - acc: 0.9894 - val_loss: 0.9380 - val_acc: 0.8503
Epoch 20/20

In [19]:

InceptionV3_model.load_weights('saved_models/weights.best.InceptionV3.hdf5')

---

### 测试模型


In [28]:

InceptionV3_predictions = [np.argmax(InceptionV3_model.predict(np.expand_dims(feature,axis = 0))) for feature in test_InceptionV3]

test_accuracy = 100*np.sum(np.array(InceptionV3_predictions)==np.argmax(test_targets, axis=1))/len(InceptionV3_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 78.8278%
